In [ ]:
import scipy.stats as stats
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import correlate, correlation_lags
import scipy
import mne
from nilearn import datasets, image, plotting
import scipy.io
import re
import seaborn
import pickle
import sklearn
from sklearn.mixture import GaussianMixture

# Computing Cross Correlations w/ Shuffled Audio Signal

In [ ]:
id_eleccount = # Removed to protect patient anonymity

In [ ]:
vis_elec = pickle.load(open("../../sourcedata/vis_elec_idx.pkl", "rb"))

In [ ]:
lags = np.load("../../derivatives/results/audio-envelope/audio_lags.npy", allow_pickle=True)

### Z-Scoring Correlations

In [ ]:
g1 = np.load("tr_lum_g1.npy", allow_pickle = True)
g2 = np.load("tr_lum_g2.npy", allow_pickle = True)

In [ ]:
def cross_correlate(film_data, neural_data):
    xcorr_arr = np.zeros((neural_data.shape[0], film_data.shape[0] + neural_data.shape[1] - 1))
    lags_arr = np.zeros((neural_data.shape[0], film_data.shape[0] + neural_data.shape[1] - 1))
    for i in range(neural_data.shape[0]):
        xcorr_arr[i,:] = correlate(film_data - np.mean(film_data), neural_data[i, :] - np.mean(neural_data[i, :]), mode='full')
        lags_arr[i,:] = correlation_lags(film_data.shape[0], neural_data.shape[1], mode="full")
    return xcorr_arr, lags_arr

In [ ]:
# Compute Z-Score for +/- 35 seconds lag around the start of the movie
def z_score_cc(cc, film_data, neural_data, electrode_num, proper_cc, spearman_lags, zero_lag_idx):
    # At every electrode and every time point, compute mean and stddev (of shuffled cc)
    shuffled_means = np.mean(cc, axis=0)
    shuffled_stdevs = np.std(cc, axis=0)
    # Z-Scored Correlations
    z_scored_cc = (proper_cc[electrode_num, :] - shuffled_means) / shuffled_stdevs
    lags_center_arr = spearman_lags[electrode_num, zero_lag_idx - 35*512:zero_lag_idx + 35*512 + 1]
    
    return z_scored_cc, lags_center_arr

In [ ]:
def z_score_full(patient_id):
    nr_1 = np.load("../../sourcedata/tie-ranked-ecog/ecog1-" + patient_id + ".npy", allow_pickle = True)
    nr_2 = np.load("../../sourcedata/tie-ranked-ecog/ecog2-" + patient_id + ".npy", allow_pickle = True)

    # Proper Correlations (G1)
    spearman_cc, spearman_lags = cross_correlate(g1, nr_1)
    zero_lag_idx = np.where(spearman_lags[0, :] == 0)[0][0]
    proper_cc_1 = spearman_cc[:, zero_lag_idx - 35*512:zero_lag_idx + 35*512 + 1]
    
    # Compute Z-Score for each electrode
    for channel in vis_elec[patient_id]:
        cc_surrog = np.load("../../derivatives/results/luminance/TRCC/tr-cc1-" + patient_id + "-" + str(channel) + ".npy", allow_pickle = True)
        zscc_1, lags_1 = z_score_cc(cc_surrog, g1, nr_1, channel, proper_cc_1, spearman_lags, zero_lag_idx)
        np.save("../../derivatives/results/luminance/ZScoredCC/zscc1-" + patient_id + "-" + str(channel) + ".npy", zscc_1)
    
    zc_by_elec = np.zeros((len(vis_elec[patient_id]), 35*512*2+1))
    
    i = 0
    for channel in vis_elec[patient_id]:
        zc_by_elec[i, :] = np.load("../../derivatives/results/luminance/ZScoredCC/zscc1-" + patient_id + "-" + str(channel) + ".npy", allow_pickle = True)
        i += 1
    np.save("../../derivatives/results/luminance/av_across_elec_1_" + patient_id + ".npy", np.mean(zc_by_elec, axis=0)) 
    np.save("../../derivatives/results/luminance/zc_by_elec_1_" + patient_id + ".npy", zc_by_elec)  
    
    # Proper Correlations (G2)
    spearman_cc, spearman_lags = cross_correlate(g2, nr_2)
    zero_lag_idx = np.where(spearman_lags[0, :] == 0)[0][0]
    proper_cc_2 = spearman_cc[:, zero_lag_idx - 35*512:zero_lag_idx + 35*512 + 1]
    
    # Compute Z-Score for each electrode
    for channel in vis_elec[patient_id]:
        cc_surrog = np.load("../../derivatives/results/luminance/TRCC/tr-cc2-" + patient_id + "-" + str(channel) + ".npy", allow_pickle = True)
        zscc_2, lags_2 = z_score_cc(cc_surrog, g2, nr_2, channel, proper_cc_2, spearman_lags, zero_lag_idx)
        np.save("../../derivatives/results/luminance/ZScoredCC/zscc2-" + patient_id + "-" + str(channel) + ".npy", zscc_2)
    
    zc_by_elec = np.zeros((len(vis_elec[patient_id]), 35*512*2+1))
    
    i = 0
    for channel in vis_elec[patient_id]:
        zc_by_elec[i, :] = np.load("../../derivatives/results/luminance/ZScoredCC/zscc2-" + patient_id + "-" + str(channel) + ".npy", allow_pickle = True)
        i += 1
    np.save("../../derivatives/results/luminance/av_across_elec_2_" + patient_id + ".npy", np.mean(zc_by_elec, axis=0)) 
    np.save("../../derivatives/results/luminance/zc_by_elec_2_" + patient_id + ".npy", zc_by_elec) 

In [ ]:
for key in id_eleccount.keys():
    z_score_full(key)

# T-Test

In [ ]:
# Removed to protect patient anonymity

# Plotting Significant Correlations

## Visual ROI Retrieval

In [ ]:
# Retrieve the AAL atlas
aal_atlas = datasets.fetch_atlas_aal('SPM12')

In [ ]:
# ROIs: Occipital Gyrus & Calcarine Sulcus

# Find the index of the target regions in the AAL atlas map
osl_idx = aal_atlas.indices[aal_atlas.labels.index('Occipital_Sup_L')]
osr_idx = aal_atlas.indices[aal_atlas.labels.index('Occipital_Sup_R')]
oml_idx = aal_atlas.indices[aal_atlas.labels.index('Occipital_Mid_L')]
omr_idx = aal_atlas.indices[aal_atlas.labels.index('Occipital_Mid_R')]
oil_idx = aal_atlas.indices[aal_atlas.labels.index('Occipital_Inf_L')]
oir_idx = aal_atlas.indices[aal_atlas.labels.index('Occipital_Inf_R')]
cl_idx = aal_atlas.indices[aal_atlas.labels.index('Calcarine_L')]
cr_idx = aal_atlas.indices[aal_atlas.labels.index('Calcarine_R')]

# Select only voxels for which the value is equal to this index
osl_map = image.math_img('img == %s' % osl_idx, img=aal_atlas.maps)
osr_map = image.math_img('img == %s' % osr_idx, img=aal_atlas.maps)
oml_map = image.math_img('img == %s' % oml_idx, img=aal_atlas.maps)
omr_map = image.math_img('img == %s' % omr_idx, img=aal_atlas.maps)
oil_map = image.math_img('img == %s' % oil_idx, img=aal_atlas.maps)
oir_map = image.math_img('img == %s' % oir_idx, img=aal_atlas.maps)
cl_map = image.math_img('img == %s' % cl_idx, img=aal_atlas.maps)
cr_map = image.math_img('img == %s' % cr_idx, img=aal_atlas.maps)


mask = image.math_img('img1 + img2 + img3 + img4 + img5 + img6 + img7 + img8', img1=osl_map, img2=osr_map, 
                      img3=oml_map, img4=omr_map, img5=oil_map, img6=oir_map, img7=cl_map, img8=cr_map)

fig = plotting.plot_roi(mask, draw_cross=False)
plt.savefig("visual_roi.png", dpi=600)

## Plotting Z-Scores For Significant Electrodes

In [ ]:
def plot_sensitive_elec(patient_id, channel_num, pos, lag):
    zc_1 = np.load("../../derivatives/results/luminance/zc_by_elec_1_" + patient_id + ".npy", allow_pickle = True)[channel_num, 15872-4*512:15872+4*512+1]
    zc_2 = np.load("../../derivatives/results/luminance/zc_by_elec_2_" + patient_id + ".npy", allow_pickle = True)[channel_num, 15872-4*512:15872+4*512+1]

    fig, ax = plt.subplots()
    plt.ylim([0.5, 2.5])
    plt.xlabel('Time From Movie Onset (ms)')
    plt.ylabel('Z-Score')
    if pos:
        plt.plot(lags[15872-4*512:15872+4*512+1]*1000/512, (zc_1 + zc_2) / 2)
    else:
        plt.plot(lags[15872-4*512:15872+4*512+1]*-1000/512, (zc_1 + zc_2) / 2)

    plt.annotate(str(lag) + ' ms', (.6, .8), xycoords='axes fraction', fontsize=15)
    ax.hlines(y=1.65, xmin=-4000, xmax=4000, linewidth=2, color='purple', linestyle='dashed')
    fig.savefig("ZCPlots/" + patient_id + "_" + str(channel_num) + ".png", dpi=600)
    plt.close(fig)

In [ ]:
# Determining significant electrodes: peak z-score greater than 1.65 percentile z-scores w/in 400 ms of movie onset

elecplos_list = []
lags_list = []

# Get significant electrode lag + MNI coordinate + plot
for key in id_eleccount:
    zc_1 = np.load("../../derivatives/results/luminance/zc_by_elec_1_" + key + ".npy", allow_pickle = True)[:, 15872-10*512:15872+10*512+1]
    zc_2 = np.load("../../derivatives/results/luminance/zc_by_elec_2_" + key + ".npy", allow_pickle = True)[:, 15872-10*512:15872+10*512+1]
    elecpos = scipy.io.loadmat('/scratch/gpfs/mayaar/GravityECoG/derivatives/preprocessing/sub-ny' + key + '/eeg1_manualica_notch_elec_elecpos.mat')['trial']
    
    for i in range(len(vis_elec[key])):
        avg = (zc_1[i, :] + zc_2[i, :]) / 2
        if (np.amax(avg) > 1.65) and (np.argmax(avg) > 4600) and (np.argmax(avg) < 5400):
            print(key, ": ", i)
            lag = (np.argmax(avg)-5000) * 1000/512
            if (lag < 0):
                plot_sensitive_elec(key, i, False, abs(lag))
            else:
                plot_sensitive_elec(key, i, True, abs(lag))
            elecplos_list.append(elecpos[vis_elec[key][i], :])
            lags_list.append((np.argmax(avg)-5000) * 1000/512)            

In [ ]:
# Manually remove non-clearly responsive electo
indices = (0,1,3,4,5,6,12,16,17,19)

elecplos_list_filtered = [elecplos_list[i] for i in indices] 
lags_list_filtered = [abs(lags_list[i]) for i in indices]

In [ ]:
lags_list_filtered

In [ ]:
np.mean(lags_list_filtered)

In [ ]:
np.std(lags_list_filtered)

In [ ]:
# Coordinates in MNI Space
og_1 = np.array([-17,-84,28])
og_2 = np.array([24,-81,31])
og_3 = np.array([-32,-81,16])
og_4 = np.array([37,-80,19])
og_5 = np.array([-36,-78,-8])
og_6 = np.array([38,-82,-8])

cal_1 = np.array([-7,-79,6])
cal_2 = np.array([16,-73,9])

vis_full = [cal_1, cal_2]

def calc_dist_from_occip(coord):
    dists = []
    
    for posit in vis_full:
        dists.append(np.linalg.norm(posit - coord))
        
    return min(dists)

In [ ]:
dists = []

for pos in elecplos_list_filtered:
    dists.append(calc_dist_from_occip(pos))

In [ ]:
dists

In [ ]:
np.mean(dists)

In [ ]:
np.std(dists)

In [ ]:
fig = plt.figure(figsize=(12, 6))
plotting.plot_markers(lags_list_filtered,
                elecplos_list_filtered,
                node_size=10,
                node_vmin=0,
                node_vmax=np.max(lags_list_filtered),
                node_cmap=plt.cm.jet,
                node_threshold=None,
                display_mode='lzry',
                figure=fig,
                axes=None,
                title=None,
                annotate=True,
                colorbar=True)
plt.savefig("lum_zc.png", dpi=600)